# Updating Plasma and Convergence

As light travels through a real plasma, it has effects on the properties of the plasma due to light-matter
interactions as well as the presence of extra energy from the light. Additionally, as [previously discussed](../montecarlo/propagation.rst), properties of the plasma affect how light travels through it. Things like this (where two things
both affect each other on different ways) frequently occur in physics. The solution is finding a steady-state for
the plasma properties; that is, the actual plasma will be in a state such that the plasma state will not change as
light propagates through it, because the effects of the light on the plasma and the effects of the plasma on the
light have been "balanced out."

One of TARDIS's main goals is to determine this plasma state (as we need the actual plasma properties in order to
get an accurate spectrum). This is done in an iterative process. After each [Monte Carlo iteration](../montecarlo/index.rst) (which sends light through the supernova ejecta), TARDIS uses the Monte Carlo estimators???
how the propagating light affects the plasma state, after which the plasma state is updated (as will be explained below and demonstrated in the code example). We do this many times, and attempt to have the plasma state converge
to the steady-state we are looking for. In fact, all but the last Monte Carlo iteration is used for this purpose
(after which TARDIS will have the needed plasma state for its last iteration which calculates the spectrum).

<div class="alert alert-info">

Note

For all but the last iteration, TARDIS uses the number of Monte Carlo packets specified in the
[Monte Carlo configuration](../../io/configuration/components/montecarlo.rst) under the ``no_of_packets`` argument. This is because
a different number of packets may be necessary to calculate the spectrum as opposed to calculate the
plasma state.

</div>

After each iteration the values for radiative temperature and dilution factor are updated by calling the ``advance_state`` method on a ``Simulation`` object as will be shown below in the code example. The goal of this is to eventually have the radiative temperature and dilution factor converge to a single value so that the steady-state plasma state can be determined. To ensure that the simulation converges, TARDIS employs additional convergence strategies. Currently, only one convergence strategy is available: damped convergence. This will be described in the following sections.

<div class="alert alert-info">

Note
    
Unless otherwise noted, all user-supplied quantities mentioned on this page are supplied in the [convergence section of the Monte Carlo configuration](../../io/configuration/components/montecarlo.rst#damped-convergence-strategy), which will be referenced as the convergence configuration.

</div>

## T_rad and W

The main way in which the plasma state is updated is by using the ``J`` and ``nu_bar`` estimators??? to update two of the key plasma inputs???: the radiative temperature $T_\mathrm{rad}$ and dilution factor $W$. Recall??? that each of these quantities takes on a different value in each shell.

Using the estimators $J$ and $\bar \nu$, we estimate these quantities using

$$T_{\mathrm{rad\ estimated}} = \frac{h}{k_{\mathrm{B}}} \frac{\pi^4}{360 \zeta(5)} \frac{\bar \nu}{J}$$

and

$$W_\mathrm{estimated} = \frac{\pi J}{\sigma_{\mathrm{R}} T_{\mathrm{rad}}^4}$$

where $h$ is Planck's constant, $k_{\mathrm{B}}$ is Boltzmann's constant, $\sigma_{\mathrm{R}}$ is the Stefan–Boltzmann constant, and $\zeta$ is the Riemann zeta function. The equation for $W$ comes from the fact that the dilution factor is the ratio of the actual mean intensity to that of a blackbody, which is $J_{\mathrm{blackbody}}=\frac{\sigma_{\mathrm{R}} T^4}{\pi}$.

Recall???, however, that when TARDIS stores these estimators, it leaves out the prefactor of $\frac{1}{4\pi V\Delta t}$ for computational ease. That is, $J=\frac{1}{4\pi V\Delta t}\sum_i \varepsilon_i l_i=\frac{1}{4\pi V\Delta t}*\mathrm{real\ J\ estimator}$, and $\bar \nu=\frac{1}{4\pi V\Delta t}\sum_i \varepsilon_i \nu_i l_i=\frac{1}{4\pi V\Delta t}*\mathrm{real\ nu\_ bar\ estimator}$. These factors are then included in our calculations; specifically, using the previous relations we have

$$T_\mathrm{rad\ estimated}= \frac{h}{k_{\mathrm{B}}} \frac{\pi^4}{360 \zeta(5)} \frac{\sum_i \varepsilon_i \nu_i l_i}{\sum_i \varepsilon_i l_i} = \frac{h}{k_{\mathrm{B}}} \frac{\pi^4}{360 \zeta(5)} \frac{\mathrm{real\ nu\_ bar\ estimator}}{\mathrm{real\ J\ estimator}}$$
and
$$W_\mathrm{estimated} = \frac{\sum_i \varepsilon_i l_i}{4\sigma_{\mathrm{R}} T_{\mathrm{rad}}^4V\Delta t} = \frac{\mathrm{real\ J\ estimator}}{4\sigma_{\mathrm{R}} T_{\mathrm{rad}}^4V\Delta t}.$$

While TARDIS can then update the plasma state using the estimated values, there is a good chance that these estimated values would “overshoot” the true value we want to converge to (for example, if the current value of the dilution factor in some cell the dilution factor is .4, and the true steady-state value TARDIS wants to find is .45, there is a good chance that the estimated value will be greater than .45). This could make the simulation take longer to converge or, at worst, make it so the simulation does not converge at all. To account for this, users can set (in the convergence configuration) a "damping constant" for both the radiative temperature ($d_{T_\mathrm{rad}}$) and the dilution factor ($d_W$). When ``advance_state`` is called, these quantities update as follows:

$$T_\mathrm{rad\ updated} = T_\mathrm{rad\ current} + d_{T_\mathrm{rad}}(T_\mathrm{rad\ estimated}-T_\mathrm{rad\ current})$$
    
and
    
$$ W_\mathrm{updated} = W_\mathrm{current} + d_W(W_\mathrm{estimated}-W_\mathrm{current}).$$

This means, for example, if the damping constant is .5, the updated value is halfway between the current value and the estimated value. If the damping constant is .7, the updated value is 70% of the way between the current value and the estimated value, and so on. **If the damping constant is 1, then the updated value is exactly the estimated value, and if the damping constant is zero, the value stays the same throughout the simulation and is not updated.**

The updated $T_\mathrm{rad}$ and $W$ are then used as inputs to the updated [plasma calculations](../setup/plasma/index.rst) which account for the effect of the Monte Carlo packets on the plasma state.

## T_inner

The temperature of the inner boundary, $T_\mathrm{inner}$, plays a unique role in the simulation, as it is the primary determiner of the output luminosity. This is because the the luminosity of the inner boundary is proportional to $T_\mathrm{inner}^4$ (see [Energy Packet Initialization](../montecarlo/initialization.ipynb)). Thus, $T_\mathrm{inner}$ is updated throughout the simulation in order to match the output luminosity to the requested luminosity specified in the [supernova configuration](../../io/configuration/components/supernova.rst) between the bounds specified in the supernova configuration. However, there is not necessarily a quartic relationship between $T_\mathrm{inner}$ and the output luminosity, as changing $T_\mathrm{inner}$ also changes the frequency distribution of the initialized packets (once again see [Energy Packet Initialization](../montecarlo/initialization.ipynb)). This then affects the light-matter interactions, affecting which packets make it to the outer boundary, which also affects the output luminosity. Because of this, there is not an exact way to estimate $T_\mathrm{inner}$. To do this estimation, we use

$$T_\mathrm{inner\ estimated} = T_\mathrm{inner\ current} * \left(\frac{L_\mathrm{output}}{L_\mathrm{requested}}\right)^{\mathrm{t\_inner\_update\_exponent}}$$
    
where $L_\mathrm{output}$ is the output luminosity calculated by adding up the luminosity of each packet (see [Basic Spectrum Generation](../spectrum/basic.ipynb)) between the bounds specified in the [supernova configuration](../../io/configuration/components/supernova.rst), $L_\mathrm{requested}$ is the luminosity requested also in the supernova configuration (requested between those bounds previously mentioned), and ``t_inner_update_exponent`` is provided by the user in the convergence configuration. Note that what we are doing is "correcting" the previous value of the inner temperature by a factor of $\left(\frac{L_\mathrm{output}}{L_\mathrm{requested}}\right)^{\mathrm{t\_inner\_update\_exponent}}$. Note that if $\frac{L_\mathrm{output}}{L_\mathrm{requested}}$ is greater than 1, we want to lower $T_\mathrm{inner}$ as the output luminosity is too high, and vice versa if the ratio is less than 1. Thus ``t_inner_update_exponent`` should be negative. Naively one might set ``t_inner_update_exponent=-0.25``, however as a default TARDIS uses ``t_inner_update_exponent=-0.5`` as -0.25 may undershoot the correct $T_\mathrm{inner}$ because of its previously mentioned effects on the initial frequency distribution.

After calculating the estimated $T_\mathrm{inner}$, the quantity is updated using damped convergence with its own damping constant (once again set in the convergence configuration):

$$T_\mathrm{inner\ updated} = T_\mathrm{inner\ current} + d_{T_\mathrm{inner}}(T_\mathrm{inner\ estimated}-T_\mathrm{inner\ current}).$$

Once again, If the damping constant is 1, then the updated value is exactly the estimated value, and if the damping constant is zero, the value stays the same throughout the simulation and is not updated.

Additionally, because of the vast impact of $T_\mathrm{inner}$ on the simulation, one may want to update it less frequently -- i.e. allow $W$ and $T_\mathrm{rad}$ to reach a steady-state value for a particular $T_\mathrm{inner}$ before updating $T_\mathrm{inner}$. To do this, in the convergence configuration we set ``lock_t_inner_cycles``, which is the number of iterations to wait before updating $T_\mathrm{inner}$. It is set to 1 by default, meaning $T_\mathrm{inner}$ would be updated every iteration.

## Convergence Information

During the simulation, information about the how $T_\mathrm{rad}$, $W$, and $T_\mathrm{inner}$ are updated as well as a comparison of the total output luminosity and the requested luminosity are logged at the INFO level (see [Configuring the Logging Output for TARDIS](../../io/optional/logging_configuration.ipynb)) as shown in the code below, to give users a better idea of how the convergence process is working.

In addition, TARDIS allows for the displaying of convergence plots, which allows users to visualize the convergence process for $T_\mathrm{rad}$, $W$, $T_\mathrm{inner}$, and the total luminosity of the supernova being modeled. For more information, see [Convergence Plots](../../io/visualization/convergence_plot.ipynb).

## Convergence Criteria

TARDIS also allows users to stop the simulation if the simulation reaches a certain level of convergence, which is checked upon the call of the ``advance_state`` method. To enable this, users must set ``stop_if_converged=True`` in the convergence configuration. Also in the configuration, the quantities ``hold_iterations``, ``threshold``, and ``fraction`` are be specified to determine convergence as follows:

For the simulation to be considered to have converged, for ``hold_iterations`` successive iterations, the estimated values of $T_\mathrm{rad}$, $W$, and $T_\mathrm{inner}$ may differ from the previous value by a fraction of at most ``threshold`` in at least ``fraction`` fraction of the shells (for $T_\mathrm{inner}$, since there is only one value, the ``fraction`` part does not apply). For example, if ``hold_iterations=3``, ``threshold=0.05`` for all three quantities, and ``fraction=.8``, the simulation will be considered to have converged if for 3 successive iterations the estimated values of $T_\mathrm{rad}$ and $W$ differ from the current respective values by at most 5% in at least 80% of the shells, *and* the estimated $T_\mathrm{inner}$ differs by at most 5%. See the [convergence configuration schema](../../io/configuration/components/montecarlo.rst#damped-convergence-strategy) for default values of these quantities.

<div class="alert alert-info">

Note
    
To determine convergence, we compare the estimated value, **not** the updated value (which is related to the estimated value via the damping constant), with the previous value. If $T_\mathrm{inner}$ is locked (see the previous section), the estimated value will still be calculated so convergence can be checked as usual.

</div>

<div class="alert alert-info">

Note
    
``hold_iterations`` and ``fraction`` are universal quantities, i.e. they are each a single value that applies to $T_\mathrm{rad}$ and $W$, and for ``hold_iterations`` also $T_\mathrm{inner}$. ``threshold``, on the other hand, is supplied for each quantity separately, so for instance you could require $T_\mathrm{rad}$ to differ by less than 1%, $W$ to differ by less than 3%, and $T_\mathrm{inner}$ to differ by less than 5% for convergence to be reached.

</div>


## Updating Other Quantities

Other quantities in the plasma state can depend on other estimators. Currently, this is only implemented for the ``J_blue`` estimator: If ``radiative_rates_type`` in the [plasma configuration](../../io/configuration/components/plasma.rst) is set to ``detailed``, the `J_blue` plasma property will will be replaced with the value of the $J^b_{lu}$ estimator??? (the raw estimator times the factor of $\frac{ct_\mathrm{explosion}}{4\pi V \Delta t}$, once again see ???), which would then affect other plasma properties that depend on the `J_blue` values (see ???). Otherwise, the `J_blue` values in the plasma are calculated as they typically are in the plasma calculations, and the $J^b_{lu}$ estimator is only used for the [formal integral](../spectrum/formal_integral.rst). Even in the former case, while the estimator does contribute to the updating of the plasma when the ``advance_state`` method is called, it does **not** contribute to the determination of if the simulation has converged, and it does **not** show up in the convergence logging or convergence plots.

## Code Example

We now show a detailed example of how the plasma is updated using the estimators after a Monte Carlo iteration. First, we import the needed packages and set up a simulation (see [Setting Up the Simulation](../setup/index.rst)):

In [ ]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.io.atom_data.util import download_atom_data
import numpy as np

from astropy import units as u, constants as const

# We download the atomic data needed to run this notebook
download_atom_data('kurucz_cd23_chianti_H_He')

In [ ]:
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)

model = sim.model
plasma = sim.plasma
runner = sim.runner

We show the initial radiative temperature, dilution factor, electron densities, and tau sobolevs in each shell:

In [ ]:
model.t_rad

In [ ]:
model.w

In [ ]:
plasma.electron_densities

In [ ]:
plasma.tau_sobolevs

We set the number of packets and we run one iteration of the Monte Carlo simulation:

In [ ]:
N_packets = 10000

# Using the commented out code below, we can also get the number of packets
# from the configuration -- try it out:
#N_packets = tardis_config.no_of_packets

sim.iterate(N_packets)

We now show the values of the three estimators previously mentioned (note that these are the raw estimators, and the factors of $\frac{1}{4\pi V \Delta t}$ etc are not included):

In [ ]:
runner.j_estimator

In [ ]:
runner.nu_bar_estimator

In [ ]:
# Because most rows of the j_blue estimatior are partially or mostly
# zero, we show just rows with all nonzero elements
runner.j_blue_estimator[np.all(runner.j_blue_estimator != 0, axis=1)]

We note that the shape of the j_blue estimator and the tau_sobolevs are the same: namely, each contain a value for each possible atomic line transition in each radial cell (as opposed to the other two estimators which just have one value for each cell):

In [ ]:
plasma.tau_sobolevs.shape, runner.j_blue_estimator.shape

We now advance the state of the simulation based on the estimators, and demonstrate this by showing the four quantities we showed before running the simulation. Compare them with their values above!

In [ ]:
# When advance_state is called, a brief summary of the updated t_rad's
# and w's is given
sim.advance_state();

In [ ]:
model.t_rad

In [ ]:
model.w

In [ ]:
plasma.electron_densities

In [ ]:
plasma.tau_sobolevs